In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import hour, dayofweek, month, date_trunc, col, split
import os

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

24/08/21 21:41:55 WARN Utils: Your hostname, coldbrew.local resolves to a loopback address: 127.0.0.1; using 172.16.119.2 instead (on interface en0)
24/08/21 21:41:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/21 21:41:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet("../data/landing/weather.parquet")

In [4]:
df = df[['DATE', 'WND', 'VIS', 'TMP', 'DEW']]
df.head()

Row(DATE='2022-01-01T00:15:00', WND='240,5,N,0026,5', VIS='011265,5,N,5', TMP='+0106,5', DEW='+0078,5')

In [5]:
df = df.withColumn("DATE", date_trunc("hour", col("DATE")))
df = df.withColumn("WND", split(col("WND"), ",").getItem(0).cast("int"))
df = df.withColumn("VIS", split(col("VIS"), ",").getItem(0).cast("int"))
df = df.withColumn("TMP", split(col("TMP"), ",").getItem(0).cast("int"))
df = df.withColumn("DEW", split(col("DEW"), ",").getItem(0).cast("int"))

In [6]:
df.head(10)

[Row(DATE=datetime.datetime(2022, 1, 1, 11, 0), WND=240, VIS=11265, TMP=106, DEW=78),
 Row(DATE=datetime.datetime(2022, 1, 1, 11, 0), WND=190, VIS=11265, TMP=106, DEW=78),
 Row(DATE=datetime.datetime(2022, 1, 1, 12, 0), WND=250, VIS=9656, TMP=110, DEW=80),
 Row(DATE=datetime.datetime(2022, 1, 1, 12, 0), WND=260, VIS=9656, TMP=106, DEW=78),
 Row(DATE=datetime.datetime(2022, 1, 1, 13, 0), WND=999, VIS=12875, TMP=111, DEW=78),
 Row(DATE=datetime.datetime(2022, 1, 1, 14, 0), WND=999, VIS=11265, TMP=106, DEW=83),
 Row(DATE=datetime.datetime(2022, 1, 1, 15, 0), WND=999, VIS=12875, TMP=110, DEW=80),
 Row(DATE=datetime.datetime(2022, 1, 1, 15, 0), WND=999, VIS=12875, TMP=106, DEW=83),
 Row(DATE=datetime.datetime(2022, 1, 1, 15, 0), WND=999, VIS=999999, TMP=9999, DEW=9999),
 Row(DATE=datetime.datetime(2022, 1, 1, 15, 0), WND=999, VIS=999999, TMP=9999, DEW=9999)]

In [7]:
output_relative_dir = "../data/raw/"
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

In [8]:
#df.to_parquet(f"{output_relative_dir}weather.parquet")